In [15]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j langchain_experimental

In [16]:
## Graphdb configuration
NEO4J_URI=""
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD=""

In [17]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [18]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [19]:
groq_api_key=""

In [20]:
from langchain_groq import ChatGroq
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-70b-versatile")

#initializing the LLMTransformer model
llm_transformer = LLMGraphTransformer(llm=llm)

#transforming documents
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mr. Dursley', type='Person'), Node(id='Mrs. Dursley', type='Person'), Node(id='Dudley Dursley', type='Person'), Node(id='Mrs. Potter', type='Person'), Node(id='Mr. Potter', type='Person')]
Relationships:[Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Mrs. Dursley', type='Person'), type='SPOUSE'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Dudley Dursley', type='Person'), type='PARENT'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='Dudley Dursley', type='Person'), type='PARENT'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='Mrs. Potter', type='Person'), type='SISTER'), Relationship(source=Node(id='Mrs. Potter', type='Person'), target=Node(id='Mr. Potter', type='Person'), type='SPOUSE')]


In [21]:
graph.add_graph_documents(
  graph_documents,
  baseEntityLabel=True,
  include_source=True
)

In [23]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "Who is the parent of id: Mr. Dursley"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: "Mr. Dursley"})-[:PARENT]->(parent:Person) RETURN parent.id
Full Context:
[{'parent.id': 'Dudley'}, {'parent.id': 'Dudley Dursley'}]

> Finished chain.


{'query': 'Who is the parent of id: Mr. Dursley',
 'result': 'Dudley is the parent of id: Mr. Dursley'}

**Using OpenAI**

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = "2023-07-01-preview"

In [ ]:
llm = AzureChatOpenAI(
    model="gpt-4",
    azure_deployment="gpt-4",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

In [ ]:
#initializing the LLMTransformer model
llm_transformer = LLMGraphTransformer(llm=llm)

#transforming documents
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    openai_api_version=api_version,
)

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
query = "Who is Dudley?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is Mr. Dursley's job?"})
response

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

result = qa_chain({"query": "What is Mr. Dursley's job?"})
result["result"]